## Coinbase Pro API using cbpro

In [10]:
import cbpro
import pandas as pd
from pandas.io.json import json_normalize
from pymongo import MongoClient

This is good for pulling in 24 hour stats and time, but I'm thinking maybe
to use historic rates to get a more precisie time

In [11]:
public_client = cbpro.PublicClient()

### Using stats 24 hour stats and time

In [12]:
stats_24hrs = public_client.get_product_24hr_stats('BTC-USD')
time_stats = public_client.get_time()

In [13]:
df_24hrs = json_normalize(stats_24hrs)
df_time = json_normalize(time_stats)

In [14]:
df_stats = pd.concat([df_time, df_24hrs], axis = 1)
df_stats

,iso,epoch,open,high,low,volume,last,volume_30day
0,2020-08-19T19:26:18.937Z,1.597865e+09,11989.34,12099.99,11611.75,16342.34176153,11704.29,484319.99809832


### Using Historic Rates

In [6]:
help(public_client.get_product_historic_rates)

Help on method get_product_historic_rates in module cbpro.public_client:

get_product_historic_rates(product_id, start=None, end=None, granularity=None) method of cbpro.public_client.PublicClient instance
    Historic rates for a product.
    
    Rates are returned in grouped buckets based on requested
    `granularity`. If start, end, and granularity aren't provided,
    the exchange will assume some (currently unknown) default values.
    
    Historical rate data may be incomplete. No data is published for
    intervals where there are no ticks.
    
    **Caution**: Historical rates should not be polled frequently.
    If you need real-time information, use the trade and book
    endpoints along with the websocket feed.
    
    The maximum number of data points for a single request is 200
    candles. If your selection of start/end time and granularity
    will result in more than 200 data points, your request will be
    rejected. If you wish to retrieve fine granularity data ov

Maybe later create a function that will automatically help us grab historical day from
the day before by generating an automatic ISO 8601 time start and end, instead of doing it manually

In [7]:
#Time start and end in ISO 8601 format
#Granularity refers to the desired timeslice in seconds. So 3600 secs is one hour

#This is for UTC time for sunday, let me know if you need it for another time zone, maybe a feature that can be added to the dashboard
ISO_start = "2020-08-16T00:00:00Z"
ISO_end = "2020-08-16T23:59:00Z"

In [8]:
#this is historical data specfically for this past sunday 08/16/2020, for each hour in UTC
bit_hist = public_client.get_product_historic_rates('BTC-USD',start = ISO_start, end = ISO_end, granularity=3600)

In [9]:
df_bit_hist = pd.DataFrame(bit_hist, columns = ['time','low','high','open','close','volume'])
df_bit_hist

,time,low,high,open,close,volume
0,1597618800,11873.83,11923.86,11909.99,11918.84,390.007840
1,1597615200,11835.01,11910.00,11861.60,11910.00,311.552199
2,1597611600,11845.00,11898.43,11849.92,11861.61,298.190880
3,1597608000,11799.33,11853.00,11824.45,11849.92,425.725707
4,1597604400,11820.00,11863.60,11835.08,11824.46,256.957096
5,1597600800,11809.70,11843.93,11837.93,11837.93,312.037200
6,1597597200,11828.68,11877.04,11867.26,11837.92,298.357453
7,1597593600,11856.43,11900.00,11866.01,11867.25,533.989672
8,1597590000,11795.62,11887.00,11811.96,11866.01,446.173132
9,1597586400,11790.74,11841.45,11818.29,11811.86,371.533772


### Using trade and book endpoints along with websocket feed to get real-time information

### using get product order book

In [10]:
#help(public_client.get_product_order_book)

In [11]:
#level 1: only the best bid and ask
#level 2: top 50 bids and asks
#level 3: full order book (non-aggregated)
public_client.get_product_order_book('BTC-USD', level = 1)


{'bids': [['11989.99', '2.16326387', 6]],
 'asks': [['11990', '0.87307797', 1]],
 'sequence': 15802671392}

#### Subscribing to websocket feed

This approach works but it's difficult to use mongo database to store responses, I will try a different approach

In [12]:
#help(cbpro.WebsocketClient)

Will be using the ticker channel, this gives real-time price updates every time a match happens.

In [13]:
mongo_client = MongoClient('mongodb://localhost:27017/')

# specify the database and collection
db = mongo_client.cryptocurrency_database
BTC_collection = db.BTC_collection

In [15]:
wsClient = cbpro.WebsocketClient(url='wss://ws-feed.pro.coinbase.com',products="BTC-USD",channels=["ticker"])
db = wsClient.start()

-- Subscribed! --

{'type': 'subscriptions', 'channels': [{'name': 'ticker', 'product_ids': ['BTC-USD']}]}
{'type': 'ticker', 'sequence': 15823280339, 'product_id': 'BTC-USD', 'price': '11700.74', 'open_24h': '11989.34', 'volume_24h': '16344.71977623', 'low_24h': '11611.75', 'high_24h': '12099.99', 'volume_30d': '484322.37611302', 'best_bid': '11700.00', 'best_ask': '11700.74', 'side': 'buy', 'time': '2020-08-19T19:26:28.451776Z', 'trade_id': 100717050, 'last_size': '0.01250089'}
{'type': 'ticker', 'sequence': 15823280425, 'product_id': 'BTC-USD', 'price': '11700.73', 'open_24h': '11989.34', 'volume_24h': '16344.72385861', 'low_24h': '11611.75', 'high_24h': '12099.99', 'volume_30d': '484322.38019540', 'best_bid': '11700.00', 'best_ask': '11700.73', 'side': 'buy', 'time': '2020-08-19T19:26:28.946544Z', 'trade_id': 100717051, 'last_size': '0.00408238'}
{'type': 'ticker', 'sequence': 15823280586, 'product_id': 'BTC-USD', 'price': '11700.74', 'open_24h': '11989.34', 'volume_24h': '16344.80

In [16]:
wsClient.close()

{'type': 'ticker', 'sequence': 15823285431, 'product_id': 'BTC-USD', 'price': '11700.01', 'open_24h': '11989.34', 'volume_24h': '16346.07241157', 'low_24h': '11611.75', 'high_24h': '12099.99', 'volume_30d': '484323.72874836', 'best_bid': '11700.00', 'best_ask': '11700.01', 'side': 'buy', 'time': '2020-08-19T19:27:01.452310Z', 'trade_id': 100717095, 'last_size': '0.0129171'}
{'type': 'ticker', 'sequence': 15823285433, 'product_id': 'BTC-USD', 'price': '11700.01', 'open_24h': '11989.34', 'volume_24h': '16346.10651914', 'low_24h': '11611.75', 'high_24h': '12099.99', 'volume_30d': '484323.76285593', 'best_bid': '11700.00', 'best_ask': '11700.01', 'side': 'buy', 'time': '2020-08-19T19:27:01.452310Z', 'trade_id': 100717096, 'last_size': '0.03410757'}
{'type': 'ticker', 'sequence': 15823285577, 'product_id': 'BTC-USD', 'price': '11700.01', 'open_24h': '11989.34', 'volume_24h': '16346.11441157', 'low_24h': '11611.75', 'high_24h': '12099.99', 'volume_30d': '484323.77074836', 'best_bid': '11700.

### Using Async and Websocket libraries

I couldn't figure out how to do this so I will try to use websocket client
with threading instead.

In [27]:
import asyncio
import websockets
import json
import nest_asyncio
nest_asyncio.apply()

In [28]:
#help(websockets.connect)

In [29]:
#how to turn string to json
message = {"type": "subscribe", "product_ids":["BTC-USD"],"channels":["ticker"]}
message = json.dumps(message)
message

'{"type": "subscribe", "product_ids": ["BTC-USD"], "channels": ["ticker"]}'

In [37]:
async def coinbaseWS():
    uri = "wss://ws-feed.pro.coinbase.com"
    async with websockets.connect(uri) as websocket:
        await websocket.send(message)
        response = await websocket.recv()
        print(response)
asyncio.get_event_loop().run_until_complete(coinbaseWS())

{"type":"subscriptions","channels":[{"name":"ticker","product_ids":["BTC-USD"]}]}


In [34]:
asyncio.

<coroutine object coinbaseWS at 0x7fb2dbd885f0>

### Using Threading with Websocket

In [1]:
import json
import time
from threading import Thread
from websocket import create_connection

In [21]:
class coinbase_websocket(dataframe):
    
    def __init__(
        self,
        connection_switch = False ,
        base_URI =  "wss://ws-feed.pro.coinbase.com",
        request_message = {"type": "subscribe", "product_ids":["BTC-USD"],"channels":["ticker"]}):
        
        self.connection_switch = connection_switch
        self.main_thread = None
        self.WS_connection = None
        self.base_URI = base_URI
        self.request_message = json.dumps(request_message)
        
    def connection(self):
        self.WS_Connection = create_connection(self.base_URI)
        self.WS_Connection.send(self.request_message)
        data = self.WS_Connection.recv()
        print(json.loads(data))
        print("Connection Succesful")
    
    def stay_alive(self):
        while self.WS_Connection.connected: 
            self.WS_Connection.ping("keepalive")
            time.sleep(30)
    
    def receive_messages(self):
        #keep_alive thread is running in parallel to main_thread
        #keep_alive is now on
        self.keep_alive.start()
        
        while self.connection_switch == True:
            data = self.WS_Connection.recv()
            message = json.loads(data)
            dataframe = 
            print(message)
            ##here we will add to a data frame or data base, which ever is best suited
    
    #close function needs work, but the rest is good.       
    def close(self):
        self.connection_switch = False
        self.WS_Connection.close()
        self.keep_alive.join()
        self.main_thread.join()
        print("Connection is closed")
    
    def start(self):
        def go():
            self.connection()
            self.receive_messages()
        
        #connection_switch turned on and threads are created.
        self.connection_switch = True
        self.main_thread = Thread(target= go)
        self.keep_alive = Thread(target= self.stay_alive)
        
        #start main_thread
        self.main_thread.start()    
    

In [22]:
WS = coinbase_websocket()

In [23]:
WS.start()

{'type': 'subscriptions', 'channels': [{'name': 'ticker', 'product_ids': ['BTC-USD']}]}
Connection Succesful
{'type': 'ticker', 'sequence': 15825936716, 'product_id': 'BTC-USD', 'price': '11731.84', 'open_24h': '12053.25', 'volume_24h': '16546.67737280', 'low_24h': '11569', 'high_24h': '12053.25', 'volume_30d': '486135.85298892', 'best_bid': '11731.83', 'best_ask': '11731.84', 'side': 'buy', 'time': '2020-08-19T22:10:11.348167Z', 'trade_id': 100736937, 'last_size': '0.02095017'}
{'type': 'ticker', 'sequence': 15825936958, 'product_id': 'BTC-USD', 'price': '11731.83', 'open_24h': '12053.25', 'volume_24h': '16546.72283307', 'low_24h': '11569', 'high_24h': '12053.25', 'volume_30d': '486135.89844919', 'best_bid': '11731.83', 'best_ask': '11731.84', 'side': 'sell', 'time': '2020-08-19T22:10:13.038203Z', 'trade_id': 100736938, 'last_size': '0.04546027'}
{'type': 'ticker', 'sequence': 15825936960, 'product_id': 'BTC-USD', 'price': '11731.83', 'open_24h': '12053.25', 'volume_24h': '16546.72877

In [25]:
#WS.close()